第五题

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def get_susan_mask():
   mask=np.ones((7,7))
   mask[0,0]=0;mask[0,1]=0;mask[0,5]=0;mask[0,6]=0
   mask[1,0]=0;mask[1,6]=0
   mask[5,0]=0;mask[5,6]=0;mask[6,0]=0;mask[6,1]=0
   mask[6,5]=0;mask[6,6]=0
   return mask


def susan_corner_detect(img_src,t=10):
   susan_mask = get_susan_mask()
   img = img_src.copy()
   row_s,col_s = 3,3
   row_e,col_e = img_src.shape[0]-3,img.shape[1]-3
   n_max = 0
   n_arr=37*np.ones(img.shape) # 初始认为没有角点
   for r in range(row_s,row_e):#遍历所有行
      for c in range(col_s,col_e):#遍历所有列
         susan_zone = img[r-3:r+3+1,c-3:c+3+1]#获取矩形区域
         susan_zone = susan_zone[susan_mask!=0]#使用mask截取圆形区域
         r0 = img[r,c]
         similarity = np.exp(-((1.0*susan_zone-r0)/t)**2 )
         n=np.sum(similarity)
         if n>n_max:
            n_max = n
         n_arr[r,c] = n
   g = n_max /2
   R = np.zeros(img.shape)
   index = n_arr<g       #小于g，认为是可能的角点，越小，可能性越大
   R[index] = g-n_arr[index] # 取反，所以R越大，是角点的可能性越大

   plt.figure()
   plt.title("edge")
   plt.imshow((6.37 * n_arr).astype(np.uint8), cmap=cm.gray)
   return R


第四题

In [ ]:
from pylab import *
from numpy import *
from scipy.ndimage import filters


def compute_harris_response(im,sigma=3):
    """ Compute the Harris corner detector response function 
        for each pixel in a graylevel image. """
    
    # derivatives
    imx = zeros(im.shape)
    filters.gaussian_filter(im, (sigma,sigma), (0,1), imx)
    imy = zeros(im.shape)
    filters.gaussian_filter(im, (sigma,sigma), (1,0), imy)
    
    # compute components of the Harris matrix
    Wxx = filters.gaussian_filter(imx*imx,sigma)
    Wxy = filters.gaussian_filter(imx*imy,sigma)
    Wyy = filters.gaussian_filter(imy*imy,sigma)
    
    # determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy
    
    return Wdet / Wtr
   
    
def get_harris_points(harrisim,min_dist=10,threshold=0.1):
    """ Return corners from a Harris response image
        min_dist is the minimum number of pixels separating 
        corners and image boundary. """
    
    # find top corner candidates above a threshold
    corner_threshold = harrisim.max() * threshold
    harrisim_t = (harrisim > corner_threshold) * 1
    
    # get coordinates of candidates
    coords = array(harrisim_t.nonzero()).T
    
    # ...and their values
    candidate_values = [harrisim[c[0],c[1]] for c in coords]
    
    # sort candidates (reverse to get descending order)
    index = argsort(candidate_values)[::-1]
    
    # store allowed point locations in array
    allowed_locations = zeros(harrisim.shape)
    allowed_locations[min_dist:-min_dist,min_dist:-min_dist] = 1
    
    # select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i,0],coords[i,1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i,0]-min_dist):(coords[i,0]+min_dist), 
                        (coords[i,1]-min_dist):(coords[i,1]+min_dist)] = 0
    
    return filtered_coords
    
    
def plot_harris_points(image,filtered_coords):
    """ Plots corners found in image. """
    
    figure()
    gray()
    imshow(image)
    plot([p[1] for p in filtered_coords],
                [p[0] for p in filtered_coords],'*')
    axis('off')
    show()